In [1]:
#!pip install tensorflow keras opencv-python matplotlib scikit-learn

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
IMG_SIZE = 224  # High quality for EfficientNetB4
DATASET_PATH = 'Downloads/dataset/d'

class_names = sorted(os.listdir(DATASET_PATH))
class_to_index = {name: idx for idx, name in enumerate(class_names)}
print("Class Labels:", class_to_index)

Class Labels: {'Nail_psoriasis': 0, 'SJS-TEN': 1, 'Vitiligo': 2, 'acne': 3, 'hyperpigmentation': 4}


In [4]:
valid_extensions = ['.jpg', '.jpeg', '.png']
total_images = 0

for class_name in class_names:
    class_dir = os.path.join(DATASET_PATH, class_name)
    count = len([f for f in os.listdir(class_dir) if os.path.splitext(f)[-1].lower() in valid_extensions])
    print(f" {class_name}: {count} images")
    total_images += count

print(f"\nTotal Images: {total_images}")

 Nail_psoriasis: 2520 images
 SJS-TEN: 3164 images
 Vitiligo: 2016 images
 acne: 1148 images
 hyperpigmentation: 700 images

Total Images: 9548


In [5]:
images = []
labels = []

max_images_per_class = 300  # You can increase to 400 if memory allows
valid_extensions = ['.jpg', '.jpeg', '.png']

for class_name in class_names:
    class_dir = os.path.join(DATASET_PATH, class_name)
    count = 0

    for img_file in os.listdir(class_dir):
        if os.path.splitext(img_file)[-1].lower() not in valid_extensions:
            continue

        img_path = os.path.join(class_dir, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(class_to_index[class_name])
            count += 1

        if count >= max_images_per_class:
            break

    print(f"Loaded {count} images from class '{class_name}'")

# Convert to arrays
images = np.array(images, dtype='float32') / 255.0
labels = to_categorical(labels, num_classes=len(class_names))

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

print(f"\n Dataset ready!")
print(f"Training samples: {len(X_train)} | Validation samples: {len(X_val)}")


Loaded 300 images from class 'Nail_psoriasis'
Loaded 300 images from class 'SJS-TEN'
Loaded 300 images from class 'Vitiligo'
Loaded 300 images from class 'acne'
Loaded 300 images from class 'hyperpigmentation'

 Dataset ready!
Training samples: 1200 | Validation samples: 300


In [6]:
input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


2025-07-06 06:20:47.015796: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,055,976 (15.47 MB)

 Trainable params: 4,013,953 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [7]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.4472 - loss: 1.3335 - val_accuracy: 0.1633 - val_loss: 1.6487
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.8261 - loss: 0.5836 - val_accuracy: 0.2100 - val_loss: 1.6591
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.9124 - loss: 0.3308 - val_accuracy: 0.2233 - val_loss: 1.6916
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9577 - loss: 0.1701 - val_accuracy: 0.2233 - val_loss: 1.6045
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.9487 - loss: 0.1641 - val_accuracy: 0.3633 - val_loss: 1.4395
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9826 - loss: 0.0833 - val_accuracy: 0.3433 - val_loss: 1.6012
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9763 - loss: 0.0754 - val_accuracy: 0.5667 - val_loss: 1.2472
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.9834 - loss: 0.0757 - val_accuracy: 0.5333 - v

In [8]:
def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return np.expand_dims(img, axis=0)

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("efficientnetb1_skin_model.h5")

test_image_path = "Downloads/dataset/d/acne/1.jpg"  # Change to any test image path
img_array = load_image(test_image_path)

prediction = model.predict(img_array)[0]

for i, prob in enumerate(prediction):
    print(f"{class_names[i]}: {prob*100:.2f}%")

predicted_class = class_names[np.argmax(prediction)]
print(f"\nPredicted class: {predicted_class}")
